### 단어\_태그\_T/F(받침여부)_원단어의발음
### https://docs.google.com/spreadsheets/d/1-9blXKjtjeKZqsf4NzHeYJCrr49-nXeRF6D80udfcwY/edit#gid=4
# 명사 NNG, 동사 VV, 형용사 VA


In [1]:
from konlpy.tag import Mecab
import pickle
import re
import sys
from gensim.models import TfidfModel
from gensim.models import Word2Vec
from gensim.corpora import Dictionary
from pprint import pprint
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import mglearn
from pprint import pprint
import numpy as np
import gc
import copy
import pandas as pd


class SB_Word2Vec():    
    
    def __init__(self, morph_list):
        self.dct = Dictionary(morph_list)
        self.corpus = [self.dct.doc2bow(line) for line in morph_list]
        self.build_Word2Vec(morph_list)
    
    def make_Word2Vec(self, morph_list, size=50, window=2, min_count=10, iteration=100):
        self.em = Word2Vec(morph_list, size=size, window=window, min_count=min_count, iter=iteration)
        self.em_vocab = list(self.em.wv.vocab.keys())
        self.em_vocab_dic = {word:idx for idx, word in enumerate(self.em_vocab)}

    def make_Word2Sen_matrix(self): 
        vocab_size = len(self.em_vocab)
        self.sen_matrix = np.zeros((len(self.corpus), vocab_size))
        for idx, row in enumerate(self.sen_matrix):
            for idx2, frequency in self.corpus[idx]:
                    if self.dct[idx2] in self.em_vocab:
                        self.sen_matrix[idx][self.em_vocab_dic[self.dct[idx2]]] = frequency                
        self.sim_matrix = np.zeros((vocab_size, vocab_size))
        for idx, w1 in enumerate(self.em_vocab):
            for idx2, w2 in enumerate(self.em_vocab):
                self.sim_matrix[idx][idx2] =  self.em.wv.similarity(w1, w2)

        self.word2sen_matrix = np.dot(self.sim_matrix, np.transpose(self.sen_matrix))

        return self.word2sen_matrix

    def get_sim_sen(self, keyword, main_text, number=1):
        self.sim_sen_index = np.argsort(self.word2sen_matrix[self.em_vocab_dic[keyword]])
        self.most_sim_sen_index = np.argmax(self.word2sen_matrix[self.em_vocab_dic[keyword]])
        index_list = self.sim_sen_index.reshape((-1,)).tolist()
        index_list.reverse()
        
        for idx, i in enumerate(index_list[:number]):
            print(str(idx + 1))
            print(main_text[i])
        return index_list
    
    def build_Word2Vec(self, morph_list):
        self.make_Word2Vec(morph_list)
        self.make_Word2Sen_matrix()
        
        
class SB_LDA():

    def make_lda(self, morph_joined, ntopic=10, learning_method='batch', max_iter=25, random_state=0, n_words=20):        
        self.vect = CountVectorizer(max_features=10000, max_df=.15)
        self.X = self.vect.fit_transform(morph_joined)
        self.lda = LatentDirichletAllocation(n_components=ntopic, learning_method=learning_method, max_iter=max_iter, random_state=random_state)
        self.document_topics = self.lda.fit_transform(self.X)
        self.sorting = np.argsort(self.lda.components_, axis=1)[:, ::-1]
        self.feature_names = np.array(self.vect.get_feature_names())
        mglearn.tools.print_topics(topics=range(ntopic), feature_names=self.feature_names, sorting=self.sorting, topics_per_chunk=5, n_words=n_words)

    def related_doc(self, main_text_list, topic_index, number=10):
        category = np.argsort(self.document_topics[:, topic_index])[::-1]
        related_docs = []
        for i in category[:number]:
            print(i)
            print(main_text_list[i] + ".\n")
            related_docs.append((i, main_text_list[i]))
        return related_docs

class SB_Tfidf():    
    
    def __init__(self, list_morph_merged):
        self.list_morph_merged = list_morph_merged
        self.dct = Dictionary(self.list_morph_merged)
        self.corpus = [self.dct.doc2bow(line) for line in self.list_morph_merged]

    def get_tfidf(self):       
        self.model = TfidfModel(self.corpus)
        self.tfidf = []
        for i in self.corpus:
             self.tfidf.append(sorted(self.model[i], key = lambda x: x[1], reverse=True))
        self.tfidf_hangul = []
        for idx1, i in enumerate(self.tfidf):
            self.tfidf_hangul.append([(self.dct[j[0]], j[1]) for j in i])        
        
        return self.tfidf_hangul
    
def frequency(merged):
    word_count = Counter(merged)
    word_count2 = []
    for i in word_count:
        word_count2.append((i, word_count[i]))
    word_count2 = sorted(word_count2, key=lambda x: x[1], reverse = True)
    return word_count2




class Social_analysis():
    
    non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
    syn_dic = {}
    del_list = []
    
    def __init__(self):
        self.mecab = Mecab()
    
    def DB_to_table(self, DBname='intake', keyword='intake'):
        import pymssql
        import pandas.io.sql as pdsql
        import pandas as pd
        self.query = \
        """
        SELECT user_id, created_at, main_text, hashtags, comments, likes, current_url FROM instaPosting WHERE keyword = '{}'
        """.format(keyword)
        conn = pymssql.connect("intakedb.c63elkxbiwfc.us-east-2.rds.amazonaws.com:1433", "gh", "ghintake", DBname)
        self.df = pdsql.read_sql_query(self.query, con=conn)
        # df['main_text'] = df.main_text.apply(lambda x: x.replace('#',' ').translate(self.non_bmp_map))
        # df['created_at'] = df.created_at.apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
        conn.close()
        self.raw_data = self.df.as_matrix()        
    
    def pickle_to_table(self, filename):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        data = data[1:]
        for idx, i in enumerate(data):
            data[idx][2] = i[2].replace('#', ' ').translate(self.non_bmp_map)
            data[idx][3] = '/'.join(i[3])
            data[idx][4] = '/'.join(i[4])
        self.raw_data = np.array(data)
  
    def hashtags_split(self, hashtags):        
        hashtags_split = []
        for i in hashtags:
            hashtags_split.append(i.split('/'))
        
        hashtags_list = []
        
        for i in hashtags_split:
            temp = []
            for j in i:
                if self.isHangul(j):
                    t_hashtags = j.translate(self.non_bmp_map)
                    temp.append(t_hashtags)
            hashtags_list.append(temp)
        self.hashtags_list = hashtags_list
        
        return hashtags_list

    
    def add_dictionary(self, *tokenized_list):
        origin_df = 1
        try:
            origin_df = pd.read_csv("C:\\mecab\\user-dic\\intake_dic.csv", encoding='utf-8', header=None)
        except:
            print('No default intake_dic')
        keyword_list = []   
        for i in tokenized_list:
            if type(i) == list:
                for j in i:
                    j = j.split('_')
                    temp = [j[0],'' ,'' ,'' ,j[1],'*',j[2], j[3],'*','*','*','*','*']
                    keyword_list.append(temp)
            else:
                i = i.split('_')
                temp = [i[0],'','','',i[1],'*',i[2], i[3], '*','*','*','*','*']
                keyword_list.append(temp)


        keyword_df = pd.DataFrame(keyword_list)
        print(type(origin_df))
        if type(origin_df) != int:
            keyword_df = pd.concat((origin_df, keyword_df), ignore_index=True)
        else: 
            print('a')
            pass
        print(keyword_df.shape)

        keyword_df.to_csv("C:\\mecab\\user-dic\\intake_dic.csv", encoding='utf-8',index=None, header=False)    
    
        
    def morph_pos(self, text_list):
        
        morph_list = []
        
        for j in text_list:
            parsed = self.mecab.pos(j)
            temp = []
            for i in parsed:
                if self.isHangul(i[0]):
                    temp.append(i[0]+'_'+i[1])
                else: pass#print('{} 한글이 아님.'.format(i[0]))
            
            morph_list.append(temp)
        
        self.df['morph_list'] = morph_list
        return morph_list

    def filter_words(self, parsed_list):
        # 1차원 리스트를 받음.
        changed_list = list(map(lambda x: self.syn_dic.get(x, x) , parsed_list))
        deleted_list = list(filter(lambda x: x not in self.del_list, changed_list))
        return deleted_list
    
    def pos_extractor(self, parsed, exception_list=['맛', '밥', '물', '몸', '죽', '먹', '없', '있', '싫', '헬', '달']):

        noun_list = []
        adj_list = []
        verb_list = []
        nav_list = []
        for j in parsed:
            
            nav_temp = []
            n_temp = []
            adj_temp = []
            verb_temp = []
            
            for i in j:            
                i = i.split('_')
                if self.isHangul(i[0]):
                    if (len(i[0]) > 1) or (i[0] in exception_list):
                        if 'NN' in i[1]:
                            n_temp.append(i[0])
                            nav_temp.append(i[0])
                        elif 'VV'in i[1]:
                            adj_temp.append(i[0])
                            nav_temp.append(i[0])
                        elif 'VA' in i[1]:
                            verb_temp.append(i[0])
                            nav_temp.append(i[0])
                    else: pass
                        #print('{} 제외'.format(i[0]))
                else: pass#print('{} 한글이 아님.'.format(i[0]))

            nav_list.append(self.filter_words(nav_temp))
            noun_list.append(self.filter_words(n_temp))
            adj_list.append(self.filter_words(adj_temp))
            verb_list.append(self.filter_words(verb_temp))
            
            
        columns=['nav_list', 'noun_list', 'adj_list', 'verb_list']
        for i in  zip(columns, [nav_list, noun_list, adj_list, verb_list]):
            self.df[i[0]] = i[1]
            
        return [nav_list, noun_list, adj_list, verb_list] # tuple(map(lambda x: [j.split('_')[0] for j in x], [nav_list, noun_list, adj_list, verb_list]))

    
    def merge_list(self, tokenized_list):
        return [j for i in tokenized_list for j in i]

    
    def join_list(self, tokenized_list):
        joined_list = []
        for idx, i in enumerate(tokenized_list):
            joined_list.append(" ".join(i))
        return joined_list
 
    def split_list(self, untokenized_list):
        hashtag_splited = []
        for idx, i in enumerate(untokenized):
            hashtag_splited.append(i.split('/'))
            return hastag_splited
        
    '''    
    def join_underbar(self, morph_list):

        all_list = []
        post_list=[]
        for i in morph_list:
            for j in i:
                post_list.append(j[0]+'_'+j[1])
            all_list.append([(' , ').join(post_list)])
            post_list=[] 
        all_list=np.array(all_list)
        
        return all_list'''

    def word_substitute(self, dataset, sublist):
        dataset = copy.deepcopy(dataset)
        sub_book = dict()
        for i in sublist:
            for j in i['sub_words']:
                sub_book[j] = i['main']
        gc.collect()
        for n, i in enumerate(dataset):
            dataset[n] = [sub_book.get(item,item) for item in i]

        del sub_book
        gc.collect()

        return dataset
    
    def word_delete(self, dataset, del_list):
        dataset = copy.deepcopy(dataset)

        for n, line in enumerate(dataset):
             dataset[n] = [i for i in line if i not in del_list]

        return dataset

    
    def isHangul(self, text):
        encText = text
        hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', encText))
        return hanCount > 0
    
    def convert_list(self, *tokenized_list):
        input_length = len(tokenized_list)
        lists = [[] for i in range(input_length)]

        for idx, li in enumerate(tokenized_list):
            for j in li:
                lists[idx].append(['/'.join(j)])

        converted_array = np.array(lists[0])
        for idx in range(input_length):
            try:
                converted_array = np.concatenate((converted_array, lists[idx + 1]), axis=1)
            except Exception as e:
                print(e,'끝')

        return converted_array

    def make_df(self, start_array, converted_array, end_array, columns=['user_id', 'created_at', 'main_text', 'morph_list', 'nav_list', 'noun_list', 'adj_list', 'verb_list', 'hashtags', 'comments', 'likes', 'current_url']):         
        df = pd.DataFrame(np.hstack((start_array, converted_array, end_array)), index=None, columns=columns)
        return df
    
    # 키워드 리스트 중 하나라도 있는 경우
    def word_check_or(self, text, keywords):
        if any(word in text for word in keywords):
            return 1
        else: return 0

    # 키워드 리스트에 있는 단어가 모두 있는 경우
    def word_check_and(self, text, keywords):
        if all(word in text for word in keywords):
            return 1
        else:
            return 0


    def word_check(self, method, keywords, df, column_name = 'main_text',filter_TF=True):
        
        filter_TF = 1 if filter_TF == True else 0
        if method == 'and':
            df['flags'] = df[column_name].apply(lambda x: self.word_check_and(x, keywords))
            return df.loc[df['flags'] == filter_TF]

        elif method == 'or':
            df['flags'] = df[column_name].apply(lambda x: self.word_check_or(x, keywords))
            return df.loc[df['flags'] == filter_TF]
        
        else:
            print('Select method, and/or')

C:\Users\abys\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
intake = Social_analysis()

In [3]:
intake.pickle_to_table('Data/intake_list.txt')

In [4]:
intake.df = pd.DataFrame(intake.raw_data, columns=['user_id', 'created_at', 'main_text', 'hashtags', 'comments', 'likes', 'current_url'])

intake.df.loc[intake.df["user_id"] == 'intakefoods']

In [5]:
intake.df = intake.word_check('or', ['자동차', '흡기', '배기','도어락'], intake.df, 'hashtags', False)
intake.df = intake.word_check('or', ['intakefoods', 'dameulstudio', '_.ddo2', '__scarlett.k', '0.8l_korea', 'jiseung86', 'untactmarket'], intake.df, 'user_id', False)

conver

temp = df_new_intake.loc[df_new['main_text'].str.contains('맛') & df_new['main_text'].str.contains('모닝죽') & df_new['main_text'].str.contains('있')]['']
temp.describe()

In [6]:
intake.modified = intake.df.as_matrix()
intake.morph_list = intake.morph_pos(intake.modified[:, 2])

In [7]:
intake.morph_list[0]

['아침_NNG',
 '식사_NNG',
 '죽_MAG',
 '모닝죽_NNG',
 '단호박_NNG',
 '인테이크_NNG',
 '몇_MM',
 '년_NNBC',
 '째_XSN',
 '먹_VV',
 '는_ETM',
 '모닝죽_NNG',
 '아침_NNG',
 '엔_JKB+JX',
 '바쁘_VA',
 '기_ETN',
 '도_JX',
 '하_VV',
 '고_EC',
 '입맛_NNG',
 '도_JX',
 '없_VA',
 '는데_EC',
 '요거_NP',
 '하나_NR',
 '면_NNG',
 '아침_NNG',
 '뚝딱_MAG',
 '맛_NNG',
 '은_JX',
 '여러_MM',
 '종류_NNG',
 '가_JKS',
 '있_VA',
 '는데_EF',
 '다_MAG',
 '맛있_VA',
 '는데_EC',
 '굳이_MAG',
 '고르_VV',
 '자면_EC',
 '단호박_NNG',
 '이랑_JKB',
 '새로_MAG',
 '나온_VV+ETM',
 '꿀고구마_NNG',
 '가_JKS',
 '맛있_VA',
 '오_EC']

In [8]:
intake.del_list = ['아침', '식사']
intake.syn_dic['인테이크'] = '랩노쉬'

In [9]:
intake.nav_list, intake.noun_list, intake.adj_list, intake.verb_list = intake.pos_extractor(intake.morph_list)

In [10]:
intake.nav_list[0]

['모닝죽',
 '단호박',
 '랩노쉬',
 '먹',
 '모닝죽',
 '바쁘',
 '입맛',
 '없',
 '맛',
 '종류',
 '있',
 '맛있',
 '고르',
 '단호박',
 '나온',
 '꿀고구마',
 '맛있']

In [11]:
intake.noun_list[0]

['모닝죽', '단호박', '랩노쉬', '모닝죽', '입맛', '맛', '종류', '단호박', '꿀고구마']

In [17]:
intake.nav_merged = intake.merge_list(intake.nav_list)

intake.nav_frequency = frequency(intake.nav_merged)

pprint(intake.nav_frequency)

[('인테이크', 5679),
 ('다이어트', 4731),
 ('먹', 4709),
 ('식단', 2596),
 ('아침', 2414),
 ('스타그램', 1790),
 ('밀스', 1775),
 ('모닝죽', 1748),
 ('맛', 1709),
 ('식사', 1631),
 ('운동', 1563),
 ('그램', 1349),
 ('맛있', 1207),
 ('다이어터', 1202),
 ('간식', 1128),
 ('건강', 1012),
 ('있', 962),
 ('저녁', 832),
 ('일상', 826),
 ('대용', 819),
 ('오늘', 763),
 ('점심', 697),
 ('일기', 673),
 ('단호박', 579),
 ('칼로리', 522),
 ('없', 521),
 ('두유', 517),
 ('소이', 499),
 ('우유', 474),
 ('소통', 451),
 ('고구마', 438),
 ('선물', 415),
 ('밥', 404),
 ('젤리', 400),
 ('데일리', 392),
 ('조절', 370),
 ('직장', 365),
 ('곤약젤리', 351),
 ('푸드', 347),
 ('밀크', 344),
 ('식품', 340),
 ('물', 338),
 ('견과', 336),
 ('홍삼', 333),
 ('하루', 328),
 ('기록', 328),
 ('맛스타', 321),
 ('미래', 319),
 ('파워', 319),
 ('모닝', 309),
 ('비타민', 304),
 ('챙겨', 303),
 ('여자', 297),
 ('이벤트', 296),
 ('한끼', 289),
 ('밀스라이트', 284),
 ('시간', 284),
 ('몸', 268),
 ('생각', 256),
 ('마시', 254),
 ('구매', 247),
 ('쉐이크', 239),
 ('후기', 238),
 ('슈퍼', 238),
 ('쿠키', 236),
 ('가슴살', 230),
 ('제품', 223),
 ('달', 218),
 ('시작', 216),
 ('

 ('원활', 11),
 ('대로', 11),
 ('전문점', 11),
 ('알려', 11),
 ('즐거움', 11),
 ('고마워요', 11),
 ('점심때', 11),
 ('아가리', 11),
 ('바이', 11),
 ('사이', 11),
 ('식이', 11),
 ('지름', 11),
 ('케어', 11),
 ('연습', 11),
 ('갈비', 11),
 ('식판', 11),
 ('마을', 11),
 ('저녁밥', 11),
 ('니즘', 11),
 ('어쩌', 11),
 ('에어', 11),
 ('건가', 11),
 ('아웃', 11),
 ('황성주', 11),
 ('휴식', 11),
 ('실온', 11),
 ('가요', 11),
 ('터짐', 11),
 ('국가', 11),
 ('단점', 11),
 ('애용', 11),
 ('섞이', 11),
 ('돌아오', 11),
 ('헤이', 11),
 ('골프', 11),
 ('시티', 11),
 ('경험', 11),
 ('기업', 11),
 ('발행', 11),
 ('내용', 11),
 ('옥수수', 11),
 ('매일', 11),
 ('카누', 11),
 ('정원', 11),
 ('극복', 11),
 ('버티', 11),
 ('만보', 11),
 ('호르몬', 11),
 ('미역국', 11),
 ('초대', 11),
 ('호텔', 11),
 ('프리바', 11),
 ('이오', 11),
 ('포맨', 11),
 ('혈액', 11),
 ('데우', 11),
 ('드셔', 11),
 ('컬러', 11),
 ('신사', 11),
 ('로즈', 11),
 ('성인', 11),
 ('스코코', 11),
 ('인스팅트', 11),
 ('시기', 10),
 ('내리', 10),
 ('챙겨서', 10),
 ('포인트', 10),
 ('줄여', 10),
 ('체리', 10),
 ('메론', 10),
 ('구이', 10),
 ('콜라보', 10),
 ('지하철', 10),
 ('천사', 10),
 ('필수품', 10),
 ('한

 ('허기지', 3),
 ('동료', 3),
 ('포크', 3),
 ('밴쿠버', 3),
 ('우정', 3),
 ('활력', 3),
 ('장국', 3),
 ('몰라요', 3),
 ('들이', 3),
 ('배추', 3),
 ('장난', 3),
 ('마카로니', 3),
 ('강정', 3),
 ('등심', 3),
 ('토탈', 3),
 ('뿌림', 3),
 ('돼지고기', 3),
 ('모밀', 3),
 ('내려오', 3),
 ('직구', 3),
 ('영양식', 3),
 ('조사', 3),
 ('아쉬워', 3),
 ('백수', 3),
 ('설마', 3),
 ('선배', 3),
 ('하인즈', 3),
 ('백숙', 3),
 ('날짜', 3),
 ('주워', 3),
 ('방식', 3),
 ('본의', 3),
 ('소용', 3),
 ('혼합', 3),
 ('품질', 3),
 ('부업', 3),
 ('마법', 3),
 ('참으', 3),
 ('유통', 3),
 ('거짓말', 3),
 ('재우', 3),
 ('베란다', 3),
 ('무가당', 3),
 ('코크', 3),
 ('파운드', 3),
 ('새우만두', 3),
 ('미만', 3),
 ('시합', 3),
 ('공화국', 3),
 ('소맥', 3),
 ('구워', 3),
 ('올려야', 3),
 ('들렀', 3),
 ('나열', 3),
 ('치즈케이크', 3),
 ('정착', 3),
 ('라따뚜이', 3),
 ('파트너', 3),
 ('실제', 3),
 ('직행', 3),
 ('오니', 3),
 ('어린이집', 3),
 ('호두과자', 3),
 ('된장', 3),
 ('조화', 3),
 ('맥반석', 3),
 ('연기', 3),
 ('몸뚱아리', 3),
 ('파스텔', 3),
 ('바란스', 3),
 ('두릅', 3),
 ('채워줄', 3),
 ('원조', 3),
 ('소식가', 3),
 ('주황색', 3),
 ('월남', 3),
 ('스크램블', 3),
 ('이나', 3),
 ('봄맞이', 3),
 ('깨달음', 3),

 ('여럿', 2),
 ('멘탈', 2),
 ('악순환', 2),
 ('자스민', 2),
 ('동인천역', 2),
 ('연간', 2),
 ('미술', 2),
 ('막내', 2),
 ('진정', 2),
 ('류추', 2),
 ('중화', 2),
 ('다면', 2),
 ('태우', 2),
 ('유료', 2),
 ('몸내', 2),
 ('빵집', 2),
 ('초과', 2),
 ('아껴서', 2),
 ('반신욕', 2),
 ('없애', 2),
 ('체온', 2),
 ('마블', 2),
 ('레트', 2),
 ('견딜', 2),
 ('광화문역', 2),
 ('마늘빵', 2),
 ('하객', 2),
 ('물병', 2),
 ('시간대', 2),
 ('단위', 2),
 ('아까울', 2),
 ('하겐다즈', 2),
 ('메토', 2),
 ('배합', 2),
 ('며느리', 2),
 ('돌입', 2),
 ('내린', 2),
 ('속편', 2),
 ('독특', 2),
 ('술자리', 2),
 ('티라미수', 2),
 ('휘저', 2),
 ('복합', 2),
 ('버릴', 2),
 ('쫓겨', 2),
 ('지냈', 2),
 ('복제품', 2),
 ('스테이트', 2),
 ('착색료', 2),
 ('감미료', 2),
 ('콘티넨탈', 2),
 ('욕조', 2),
 ('전포동', 2),
 ('게으른', 2),
 ('다미', 2),
 ('따리', 2),
 ('종무식', 2),
 ('대하', 2),
 ('시미', 2),
 ('이브', 2),
 ('개추', 2),
 ('맘마', 2),
 ('보이차', 2),
 ('과메기', 2),
 ('하카', 2),
 ('고소미', 2),
 ('다한', 2),
 ('관계자', 2),
 ('포도즙', 2),
 ('한양대', 2),
 ('수수부꾸미', 2),
 ('장사', 2),
 ('김리아', 2),
 ('엔터', 2),
 ('스틸', 2),
 ('서핑', 2),
 ('아이패드', 2),
 ('키보드', 2),
 ('지침', 2),
 ('주택난', 2),

 ('개울가', 1),
 ('우습', 1),
 ('래드', 1),
 ('험로', 1),
 ('윈치', 1),
 ('다카르', 1),
 ('라조', 1),
 ('워밍업', 1),
 ('주무셔서', 1),
 ('광운대역', 1),
 ('나려', 1),
 ('날려서', 1),
 ('기표', 1),
 ('안사', 1),
 ('팩맨', 1),
 ('부어라', 1),
 ('버터나이프', 1),
 ('뜨거우', 1),
 ('식혀서', 1),
 ('풀떼기', 1),
 ('빡쎄', 1),
 ('베이커', 1),
 ('밀려온다', 1),
 ('글씨', 1),
 ('샘플링', 1),
 ('봄봄', 1),
 ('스페이스', 1),
 ('기여', 1),
 ('운거', 1),
 ('극인', 1),
 ('책임진다', 1),
 ('죽마고우', 1),
 ('한울', 1),
 ('기엽', 1),
 ('방부제', 1),
 ('다둥', 1),
 ('야가', 1),
 ('나아질까', 1),
 ('코엑스몰', 1),
 ('목우', 1),
 ('낯익', 1),
 ('보급', 1),
 ('설악', 1),
 ('그란', 1),
 ('연보', 1),
 ('라리시', 1),
 ('빗겨', 1),
 ('모서리', 1),
 ('반가워서', 1),
 ('나으면', 1),
 ('피할', 1),
 ('즐겨라', 1),
 ('사역', 1),
 ('피코', 1),
 ('약침', 1),
 ('부항', 1),
 ('재구', 1),
 ('호출', 1),
 ('건졌', 1),
 ('어르', 1),
 ('아까운', 1),
 ('묻혀서', 1),
 ('손질', 1),
 ('날림', 1),
 ('알콜', 1),
 ('거친', 1),
 ('세파', 1),
 ('스로', 1),
 ('호고', 1),
 ('원규', 1),
 ('멍게', 1),
 ('몰려와서', 1),
 ('튀겨', 1),
 ('불륜', 1),
 ('명소', 1),
 ('물왕', 1),
 ('저수지', 1),
 ('박원규', 1),
 ('집니다', 1),
 ('제로', 1

 ('안궁', 1),
 ('꿀돼지', 1),
 ('놀림', 1),
 ('애증', 1),
 ('퍼레이드', 1),
 ('준다는', 1),
 ('아영', 1),
 ('서운동', 1),
 ('용덕동', 1),
 ('포천시', 1),
 ('자갈치시장', 1),
 ('행당역', 1),
 ('통복동', 1),
 ('미음동', 1),
 ('선동', 1),
 ('미대동', 1),
 ('독일', 1),
 ('풍무동', 1),
 ('변동', 1),
 ('이화동', 1),
 ('스티로폼', 1),
 ('세이버', 1),
 ('겨울맞이', 1),
 ('선풍', 1),
 ('인기몰이', 1),
 ('신진', 1),
 ('예술가', 1),
 ('컬렉션', 1),
 ('침구', 1),
 ('유니크', 1),
 ('가든', 1),
 ('월동', 1),
 ('영동군', 1),
 ('황호동', 1),
 ('도봉', 1),
 ('동호동', 1),
 ('양정동', 1),
 ('서호동', 1),
 ('선바위역', 1),
 ('생용동', 1),
 ('용탄동', 1),
 ('백두대간', 1),
 ('점봉산', 1),
 ('산시', 1),
 ('경량', 1),
 ('제입맛에', 1),
 ('숙지', 1),
 ('침전물', 1),
 ('정작', 1),
 ('여가', 1),
 ('소멸', 1),
 ('꽁짜', 1),
 ('젬마', 1),
 ('냅니다', 1),
 ('원당역', 1),
 ('야자', 1),
 ('스퀘어드', 1),
 ('왕길', 1),
 ('어물동', 1),
 ('월계동', 1),
 ('깨소금', 1),
 ('유천동', 1),
 ('상천역', 1),
 ('성수역', 1),
 ('남해', 1),
 ('가거도', 1),
 ('봉계동', 1),
 ('용두역', 1),
 ('낭요', 1),
 ('팔레오', 1),
 ('유투', 1),
 ('협력', 1),
 ('서교', 1),
 ('전액', 1),
 ('서교동', 1),
 ('골격', 1),
 ('백조', 1),
 ('중간식', 1),
 ('석식',

 ('담임', 1),
 ('귀여움', 1),
 ('진정한', 1),
 ('장속', 1),
 ('배양', 1),
 ('장건', 1),
 ('비커', 1),
 ('수맛', 1),
 ('먼데이', 1),
 ('판타지', 1),
 ('해지', 1),
 ('장아', 1),
 ('겁났', 1),
 ('이달', 1),
 ('농협', 1),
 ('만리', 1),
 ('세용', 1),
 ('내릴래', 1),
 ('공진단', 1),
 ('카베', 1),
 ('식혜', 1),
 ('찾아가', 1),
 ('영양학', 1),
 ('스피룰리나', 1),
 ('금상첨화', 1),
 ('분리수거', 1),
 ('뒷주머니', 1),
 ('임박', 1),
 ('바리오', 1),
 ('동반자', 1),
 ('헤즈메', 1),
 ('리카', 1),
 ('워러', 1),
 ('법석', 1),
 ('테크', 1),
 ('선도', 1),
 ('사옥', 1),
 ('신들린', 1),
 ('리나', 1),
 ('번거롭', 1),
 ('파티션', 1),
 ('넘어오', 1),
 ('들린', 1),
 ('이이', 1),
 ('기다릴게요', 1),
 ('기다립니다', 1),
 ('까부', 1),
 ('파전', 1),
 ('동동주', 1),
 ('뜬물', 1),
 ('유익', 1),
 ('유해', 1),
 ('헝그리', 1),
 ('푸른색', 1),
 ('대리', 1),
 ('불토', 1),
 ('강동구', 1),
 ('유부남', 1),
 ('던데', 1),
 ('풀기', 1),
 ('마셔서', 1),
 ('배아', 1),
 ('방요', 1),
 ('걸어다니', 1),
 ('산고', 1),
 ('세노', 1),
 ('비스만', 1),
 ('것만', 1),
 ('난단', 1),
 ('양림동', 1),
 ('서구', 1),
 ('남구', 1),
 ('레보', 1),
 ('소미', 1),
 ('부끄러움', 1),
 ('기다릴까', 1),
 ('생고구마', 1),
 ('철분제', 1),
 ('무지하', 1),
 ('이

In [19]:
intake.nav_joined = intake.join_list(intake.nav_list)

In [7]:
intake.LDA = SB_LDA()
intake.LDA.make_lda(intake.nav_joined, ntopic=5, learning_method='batch', max_iter=25, random_state=0, n_words=20)

AttributeError: 'Social_analysis' object has no attribute 'nav_joined'

In [1]:
def add_dictionary(*tokenized_list):
    origin_df = 1
    try:
        origin_df = pd.read_csv("C:\\mecab\\user-dic\\intake_dic.csv", encoding='utf-8', header=None)
    except:
        print('No default intake_dic')
    keyword_list = []   
    for i in tokenized_list:
        if type(i) == list:
            for j in i:
                j = j.split('_')
                temp = [j[0],'' ,'' ,'' ,j[1],'',j[2], j[3],'*','*','*','*']
                keyword_list.append(temp)
        else:
            i = i.split('_')
            temp = [i[0],'','','',i[1],'',i[2], i[3], '*','*','*','*']
            keyword_list.append(temp)


    keyword_df = pd.DataFrame(keyword_list)
    print(type(origin_df))
    if type(origin_df) != int:
        keyword_df = pd.concat((origin_df, keyword_df), ignore_index=True)
    else: 
        print('a')
        pass
    print(keyword_df.shape)

    keyword_df.to_csv("C:\\mecab\\user-dic\\intake_dic.csv", encoding='utf-8',index=None, header=False)

In [4]:
import pandas as pd
add_dictionary('인테이크_NNP_F_인테이크')

No default intake_dic
<class 'int'>
a
(1, 12)


In [ ]:
단어_태그_T/F(받침여부)_원단어의발음

In [9]:
keyword_dic = '''모닝죽,,,,NNG,*,T,모닝죽,*,*,*,*,*
인테이크,,,,NNG,*,F,인테이크,*,*,*,*,*
꿀고구마,,,,NNG,*,F,꿀고구마,*,*,*,*,*
한끼,,,,NNG,*,F,한끼,*,*,*,*,*
밀스라이트,,,,NNG,*,F,밀스라이트,*,*,*,*,*
식단인증,,,,NNG,*,F,식단인증,*,*,*,*,*
직장인,,,,NNG,*,T,직장인,*,*,*,*,*
스타그램,,,,NNG,*,T,스타그램,*,*,*,*,*
귀리우유,,,,NNG,*,F,귀리우유,*,*,*,*,*
텐바이텐,,,,NNG,*,T,텐바이텐,*,*,*,*,*
텐텐쇼퍼,,,,NNG,*,F,텐텐쇼퍼,*,*,*,*,*
간편식,,,,NNG,*,T,간편식,*,*,*,*,*
1일1식,,,,NNG,*,T,1일1식,*,*,*,*,*
카페라떼,,,,NNG,*,F,카페라떼,*,*,*,*,*
벨벳초콜렛,,,,NNG,*,T,벨벳초콜렛,*,*,*,*,*
혼밥,,,,NNG,*,T,혼밥,*,*,*,*,*
곤약현미밥,,,,NNG,*,T,곤약현미밥,*,*,*,*,*
버터치킨커리,,,,NNG,*,F,버터치킨커리,*,*,*,*,*
유지어터,,,,NNG,*,F,유지어터,*,*,*,*,*
치킨커리,,,,NNG,*,F,치킨커리,*,*,*,*,*
버터치킨,,,,NNG,*,F,버터치킨,*,*,*,*,*
밀스칩,,,,NNG,*,T,밀스칩,*,*,*,*,*
미래지향,,,,NNG,*,F,미래지향,*,*,*,*,*
유통기한,,,,NNG,*,T,유통기한,*,*,*,*,*
고농축,,,,NNG,*,T,고농축,*,*,*,*,*
단백질,,,,NNG,*,T,단백질,*,*,*,*,*
밀스드링크딸기,,,,NNG,*,F,밀스드링크딸기,*,*,*,*,*
프레시모닝,,,,NNG,*,T,프레시모닝,*,*,*,*,*
대체,,,,NNG,*,F,대체,*,*,*,*,*
클랜즈,,,,NNG,*,F,클랜즈,*,*,*,*,*
와디즈,,,,NNG,*,F,와디즈,*,*,*,*,*
얼리버드,,,,NNG,*,F,얼리버드,*,*,*,*,*
까페라떼,,,,NNG,*,F,까페라떼,*,*,*,*,*
로얄밀크티,,,,NNG,*,F,로얄밀크티,*,*,*,*,*
곤약젤리,,,,NNG,*,F,곤약젤리,*,*,*,*,*
모닝채소,,,,NNG,*,F,모닝채소,*,*,*,*,*
콜드프레스,,,,NNG,*,F,콜드프레스,*,*,*,*,*
하루야채,,,,NNG,*,F,하루야채,*,*,*,*,*
씨리얼,,,,NNG,*,T,씨리얼,*,*,*,*,*
콜드부르,,,,NNG,*,F,콜드부르,*,*,*,*,*
스타벅스,,,,NNG,*,F,스타벅스,*,*,*,*,*
밀스드링크,,,,NNG,*,F,밀스드링크,*,*,*,*,*
존맛,,,,VA,*,T,존맛,*,*,*,*,*
아워홈,,,,NNG,*,T,아워홈,*,*,*,*,*
모닝바게트,,,,NNG,*,F,모닝바게트,*,*,*,*,*
모닝귀리,,,,NNG,*,F,모닝귀리,*,*,*,*,*
스윗그레인,,,,NNG,*,T,스윗그레인,*,*,*,*,*
그릭요거트,,,,NNG,*,F,그릭요거트,*,*,*,*,*
팥톡스,,,,NNG,*,F,팥톡스,*,*,*,*,*
식이섬유,,,,NNG,*,F,식이섬유,*,*,*,*,*
베지밀,,,,NNG,*,T,베지밀,*,*,*,*,*
그레인하프,,,,NNG,*,F,그레인하프,*,*,*,*,*
코코넛하프,,,,NNG,*,F,코코넛하프,*,*,*,*,*
식이조절,,,,NNG,*,T,식이조절,*,*,*,*,*
줌마,,,,NNG,*,F,줌마,*,*,*,*,*
매일유업,,,,NNG,*,T,매일유업,*,*,*,*,*
볶음콩,,,,NNG,*,T,볶음콩,*,*,*,*,*
상하목장,,,,NNG,*,T,상하목장,*,*,*,*,*
요거트,,,,NNG,*,F,요거트,*,*,*,*,*
카카오닙스,,,,NNG,*,F,카카오닙스,*,*,*,*,*
잇클린,,,,NNG,*,T,잇클린,*,*,*,*,*
카페다이어트,,,,NNG,*,F,카페다이어트,*,*,*,*,*
사과맛워터젤리,,,,NNG,*,F,사과맛워터젤리,*,*,*,*,*
맛있는다이어트칩,,,,NNG,*,T,맛있는다이어트칩,*,*,*,*,*
다이어트칩,,,,NNG,*,T,다이어트칩,*,*,*,*,*'''.split('\n')

In [10]:
for idx, i in enumerate(keyword_dic):
    keyword_dic[idx] = i.split(',')

In [11]:
pd.DataFrame(keyword_dic).to_csv('c:\\mecab\\user-dic\\intake_dic.csv', encoding='utf-8', index=None, header=False)

In [18]:
keyword_list = []

In [20]:
for idx in range(len(intake.morph_list)):
    print(intake.morph_list[idx])
    print(intake.modified[idx, 2])
    print(idx)
    k = input()    
    while k:
        keyword_list.append(k)
        k = input()
        if k == 'break' or k == 'next':
            break
    if k == 'break':
        break

['아침_NNG', '식사_NNG', '죽_MAG', '모닝죽_NNG', '단호박_NNG', '인테이크_NNG', '몇_MM', '년_NNBC', '째_XSN', '먹_VV', '는_ETM', '모닝죽_NNG', '아침_NNG', '엔_JKB+JX', '바쁘_VA', '기_ETN', '도_JX', '하_VV', '고_EC', '입맛_NNG', '도_JX', '없_VA', '는데_EC', '요거_NP', '하나_NR', '면_NNG', '아침_NNG', '뚝딱_MAG', '맛_NNG', '은_JX', '여러_MM', '종류_NNG', '가_JKS', '있_VA', '는데_EF', '다_MAG', '맛있_VA', '는데_EC', '굳이_MAG', '고르_VV', '자면_EC', '단호박_NNG', '이랑_JKB', '새로_MAG', '나온_VV+ETM', '꿀고구마_NNG', '가_JKS', '맛있_VA', '오_EC']
lora.study 아침식사 죽 모닝죽 단호박 인테이크
.
몇년째 먹는 모닝죽
아침엔 바쁘기도 하고 입맛도 없는데 요거하나면 아침뚝딱�
맛은 여러종류가 있는데.. 다 맛있는데 굳이 고르자면 단호박이랑 새로 나온 꿀고구마가 맛있오�
0

['바쁜_VA+ETM', '아침_NNG', '식사_NNG', '대용_NNG', '으로_JKB', '마시_VV', '고_EC', '있_VX', '는_ETM', '가벼운_VA+ETM', '한끼_NNG', '인테이크_NNG', '밀스라이트_NNG', '간편_XR', '하_XSA', '게_EC', '먹_VV', '을_ETM', '수_NNB', '있_VV', '고_EC', '영양소_NNG', '까지_JX', '챙길_VV+ETM', '수_NNB', '있_VV', '어요_EF', '밀스라이트_NNG', '덕분_NNG', '에_JKB', '아침_NNG', '거르_VV', '는_ETM', '일_NNG', '없_VA', '어요_EF', '아침_NNG', '마다_JX', '포만감_NNG', '있_VV', '게_EC', '호로록_MAG


['밥_NNG', '못_MAG', '먹_VV', '는_ETM', '대신_NNG', '모닝죽_NNG', '으로_JKB', '삼_NR', '시_NNBC', '세끼_NNG', '먹_VV', '고_EC', '이벤트_NNG', '까지_JX', '당첨_NNG', '됐_XSV+EP', '으면_EC', '좋_VA', '겠_EP', '다_EC']
09_21g밥 못 먹는 대신 모닝죽으로 삼시세끼 먹고 이벤트까지 당첨됐으면 좋겠다��
17

['오늘_NNG', '의_JKG', '식단_NNG', '아침_NNG', '망고_NNG', '스무디_NNP', '수박_NNG', '점심_NNG', '밀스_NNP', '계란_NNG', '개_NNBC', '밀스_NNP', '만_JX', '먹_VV', '으면_EC', '배고플_VA+ETM', '거_NNB', '같_VA', '아서_EC', '밀스_NNP', '에_JKB', '시리얼_NNP', '추가_NNG', '하고_JKQ', '계란_NNG', '개_NNBC', '다른_MM', '분_NNB', '들_XSN', '간식_NNG', '믹스_NNG', '커피_NNG', '한잔_NNG', '콜드_NNG', '브루_NNP', '저녁_NNG', '고구마_NNP', '샐러드_NNG', '닭_NNG', '가슴살_NNG', '또_MAG', '띠_VV', '아_EC', '양상추_NNG', '오이_NNG', '치즈_NNG', '장_NNG', '닭_NNG', '가슴살_NNG', '스_IC', '테_NNG', '끼_NNG', '방울토마토_NNG', '다이어트_NNG', '다이어터_NNP', '다이어트_NNG', '식단_NNG', '다이어터_NNP', '일기_NNG', '다이어트_NNG', '그램_NNG', '목요일_NNG', '필라테스_NNP', '일상_NNG', '운동_NNG', '얼른_MAG', '주말_NNG', '망고_NNG', '수박_NNG', '인테이크_NNG', '밀스_NNP', '씨_NNB', '리얼_NNG', '식단_NNG', '점심_NNG', '저녁_NNG'


['나_NP', '는_JX', '잘_MAG', '먹_VV', '고_EC', '잘_MAG', '살_VV', '고_EC', '있_VX', '는_ETM', '중_NNB', '이_VCP', '다_EC', '다이어트_NNG', '는_JX', '개_NNBC', '나_JX', '줘_VV+EC', '버려_VX+EC', '다이어트_NNG', '는_JX', '저_MM', '멀리_NNG', '로_JKB', '라고_VCP+EC', '말_NNG', '하_XSV', '고선_EC+JX', '이벤트_NNG', '하_VV', '길래_EC', '주워온_VV+EC+VX+ETM', '녀석_NNB', '들_XSN', '인테이크_NNG', '모닝귀리_NNG', '게다가_MAG', '쇼핑몰_NNG', '특가_NNG', '로_JKB', '거의_MAG', '가격_NNG', '에_JKB', '구입_NNG', '이걸_NP+JKO', '카드_NNG', '포인트_NNG', '써서_VV+EC', '샀_VV+EP', '다는_ETM', '건_NNB+JX', '안_MAG', '비밀_NNG', 'ㅋㅋ_IC', '카드_NNG', '포인트_NNG', '라는_VCP+ETM', '건_NNB+JX', '원래_MAG', '뻘_NNG', '짓_NNG', '할_XSV+ETM', '때_NNG', '쓰_VV', '라고_EC', '주_VX', '는_ETM', '거_NNB', '임_VCP+ETN', '년_NNBC', '동안_NNG', '빚_NNG', '억_NR', '갚_VV', '으며_EC', '살_VV', '기_ETN', '년_NNBC', '차_NNG']
hoonseokjeon 나는잘먹고잘살고있는중이다
 다이어트는개나줘버려  다이어트는저멀리로
라고 말하고선 3+1 이벤트 하길래 주워온 녀석들
 인테이크  INTAKE  모닝귀리  MEALS_LIGHT
게다가 쇼핑몰 특가로 거의 50% 가격에 구입
이걸  카드포인트 써서 샀다는건  안비밀 ㅋㅋ
 카드포인트 라는건 원래 뻘짓할 때 쓰라고 주는거임
 4년동안_빚1억_갚으며_살기_1년차
27




['땅_NNG', '땅_NNG', '아침_NNG', '요거트_NNG', '볼_VV+ETM', '요거트_NNG', '사과_NNG', '카카오닙스_NNG', '밀스라이트_NNG', '카카오_NNG', '조금_NNG', '얼마_NNG', '만_JX', '의_JKG', '요거_NP', '모닝_NNP', '감격_NNG', '시간_NNG', '여유_NNG', '가_JKS', '좀_MAG', '있_VV', '기_ETN', '도_JX', '하_VV', '고_EC', '요거트_NNG', '얼마_NNG', '안_MAG', '남_VV', '아서_EC', '박박_MAG', '긁_VV', '어서_EC', '만들_VV', '어_EC', '왔_VX+EP', '다_EC', '사과_NNG', '도_JX', '마지막_NNG', '남_VV', '은_ETM', '사과_NNG', '였_VCP+EP', '는데_EC', '너무_MAG', '물컹_XR', '했_XSA+EP', '음_ETN', 'ㅠㅠㅠ_UNKNOWN', '오래_MAG', '놔둬서_VV+EC+VX+EC', '그런_MM', '가부_NNG', '다_MAG', '아쉽_VA', '그치만_MAJ', '역시_MAJ', '요거트_NNG', '는_JX', '존맛_VA', '탱_MAG', '요_MM', '겉_NNG', '평생_NNG', '못_MAG', '잃_VV', '어_EF', '바니_NNP', '챌린지_NNG', '하루_NNG', '에_JKB', '물_NNG', '이상_NNG', '과식_NNG', '폭식_NNG', '금지_NNG', '자기_NP', '전_MM', '폼_NNG', '롤러_NNG', '꼭_MAG', '아침_NNG', '식단_NNG', '식단_NNG', '기록_NNG', '다이어트_NNG', '식단_NNG', '잇클린_NNG', '요거트_NNG', '볼_VV+ETM', '인테이크_NNG', '밀스라이트_NNG', '다이어트_NNG', '다이어터_NNP', '다이어트_NNG', '그램_NNG']
d_ddang2 땅180703  땅_아침�
.


['오늘_NNG', '의_JKG', '식단_NNG', '점심_NNG', '뮤_NNG', '즐_XPN', '리_NNG', '다이어트_NNG', '칩_NNG', '저녁_NNG', '에_JKB', '파_VV+EC', '틋하_UNKNOWN', '묵_VV', '으러_EC', '갈_VX+ETM', '꺼_NNB+VCP', '라_EC', '더욱_MAG', '간단_XR', '하_XSA', '게_EF', '하_IC', '저_MM', '소이_NNG', '칩_NNG', '너무너무_MAG', '맛있_VA', '다_EC', '빠삭빠삭_MAG', '하_VV', '고_EC', 'ㅎㅎ_IC', '위_NNG', '메_VV+EC', '프_VX', '에서_JKB', '세일_NNG', '할_XSV+ETM', '때_NNG', '쟁여_VV+EC', '놔야_VX+EC+VX', '징_EF', '다이어트_NNG', '다이어트_NNG', '식단_NNG', '다이어터_NNP', '다이어트_NNG', '계정_NNG', '다이어트_NNG', '계정_NNG', '인테이크_NNG', '다이어트칩_NNG', '뮤_NNG', '즐_XPN', '리_NNG', '뮤_NNG', '즐_XPN', '리_NNG', '다이어트_NNG', '우유_NNG', '건강_NNG', '건강_NNG', '하_XSV', '게_EC']
diet.goo 20180701  오늘의식단 .
.
�점심 뮤즐리+다이어트 칩 .
저녁에 파틋하 묵으러 갈꺼라 더욱 간단하게! 하 저 소이 칩 너무너무 맛있다 빠삭빠삭하고 ㅎㅎ 위메프에서 세일할때 쟁여 놔야징
.
.
.
 다이어트  다이어트식단  다이어터  다이어트계정  다이어트계정 인테이크  다이어트칩  뮤즐리  뮤즐리다이어트  우유  건강  건강하게
47



KeyboardInterrupt: 

In [ ]:
셀